In [1]:
from modify_cost_function import *
import ase.io
import parmed as pmd
from parmed import gromacs
from insertHbyList import insertHbyList

In [2]:
implicitHbondingPartners = {'CD4':1,'CD3':1,'CA2':2,'CA3':2,'CB2':2,'CB3':2}

infile_pdb = 'sandbox/horton_esp_fit_constrained/system100.pdb'
infile_top = 'sandbox/horton_esp_fit_constrained/system100.top'

ua_ase_struct = ase.io.read(infile_pdb)
ua_pmd_struct = pmd.load_file(infile_pdb)
ua_pmd_top = gromacs.GromacsTopologyFile(infile_top,parametrize=False)
# throws some warnings on angle types, does not matter for bonding info
# if error thrown, just try to "reduce" .top as far as possible
ua_pmd_top.strip(':SOL,CL') # strip water and electrolyte from system (if not yet done in .top)
ua_pmd_top.box = ua_pmd_struct.box # Needed because .pdb contains box info
ua_pmd_top.positions = ua_pmd_struct.positions

ua_names = [ a.name for a in ua_pmd_top.atoms ]
ua_residues = [ a.residue.name for a in ua_pmd_top.atoms ]

aa_ase_struct, aa_pmd_struct, aa_names, aa_residues = insertHbyList(ua_ase_struct,ua_pmd_top,
        implicitHbondingPartners,1.0)

ua_count = len(ua_ase_struct)     # united atoms structure
aa_count = len(aa_ase_struct) # all atoms structure

/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:508: GromacsWarning: bond funct != 1; unknown functional
  GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:508: GromacsWarning: bond funct != 1; unknown functional
  GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:508: GromacsWarning: bond funct != 1; unknown functional
  GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:508: GromacsWarning: bond funct != 1; unknown functional
  GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:508: GromacsWarning: bond funct != 1; unknown functional
  GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:508: GromacsWarning: bond funct != 1; unknown functional
  GromacsWarning)
/home/fr/fr_fr/f

Adding 1 H-atoms to CD3 (#8)...
bondingPartners [ 9 21]
Atom CD3 already has bonding partners CD4, CB1
Adding H-atom 1CD3 at position [ 23.911579364757202, 25.466622991293512, 12.349999999999998 ]
Adding 1 H-atoms to CD4 (#9)...
bondingPartners [10 12]
Atom CD4 already has bonding partners CD5, CA1
Adding H-atom 1CD4 at position [ 25.312471957818314, 25.072450829115194, 10.489569761035874 ]
Adding 2 H-atoms to CA2 (#15)...
bondingPartners [16]
Atom CA2 already has bonding partners CA3
Adding H-atom 1CA2 at position [ 28.74261935828807, 23.557590675857806, 11.104589188842278 ]
bondingPartners [ 16 105]
Atom CA2 already has bonding partners CA3, 1CA2
Adding H-atom 2CA2 at position [ 28.85271690173004, 23.702220854918263, 9.389826475843604 ]
Adding 2 H-atoms to CA3 (#16)...
bondingPartners [17]
Atom CA3 already has bonding partners NA1
Adding H-atom 1CA3 at position [ 30.129780545620445, 25.445820959539404, 10.879183744085134 ]
bondingPartners [ 17 107]
Atom CA3 already has bonding partne

/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:559: GromacsWarning: angles funct != 1 or 5; unknown functional
  'functional', GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:559: GromacsWarning: angles funct != 1 or 5; unknown functional
  'functional', GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:559: GromacsWarning: angles funct != 1 or 5; unknown functional
  'functional', GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:559: GromacsWarning: angles funct != 1 or 5; unknown functional
  'functional', GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/gromacstop.py:559: GromacsWarning: angles funct != 1 or 5; unknown functional
  'functional', GromacsWarning)
/home/fr/fr_fr/fr_jh1130/.local/lib/python3.6/site-packages/parmed/gromacs/

In [3]:
atom_residue_list = list(zip(aa_names,aa_residues))
ase_id = range(aa_count)
ase2pmd = dict(zip(ase_id,atom_residue_list))
pmd2ase = dict(zip(atom_residue_list,ase_id))

In [4]:
ua_index = np.arange(ua_count)
aa_index = np.arange(aa_count)

In [5]:
A_horton, B_horton, C_horton, N_horton = read_horton_cost_function(
    file_name = 'sandbox/horton_esp_fit_constrained/smamp_esp_cost_ua.h5')

In [6]:
### Charge Groups:
# read in all charge groups and construct the corresponding constraints
cg, cg_max = read_Name_Index_ChargeGroup(
    file_name = 'sandbox/horton_esp_fit_constrained/Atom_name_charge_group.csv',name=ua_names)

In [7]:
cg_max

10

In [8]:
cg_q = read_ChargeGroup_TotalCharge(
    file_name = 'sandbox/horton_esp_fit_constrained/charge_group_total_charge.csv')

In [9]:
cg_q

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 0,  0,  0,  0,  1,  0,  1,  0,  0,  0]])

In [10]:
#loop over set of charge groups (each charge group occures only ones)
c_groups = []
charges  = []
for cg_value in set(cg):
    #step with index only necessary if index is not ordered
    indices = ua_index[np.where(cg == cg_value)[0]]
    c_groups.append(indices)
    c = cg_q[1, np.where(cg_q[0] == cg_value%(cg_max+1))][0][0]
    charges.append(c)

In [11]:
c_groups

[array([0, 1, 2]),
 array([3, 4]),
 array([ 5,  6,  7,  8,  9, 10, 11]),
 array([12, 13, 14, 15]),
 array([16, 17, 18, 19, 20]),
 array([21, 22, 23, 24]),
 array([25, 26, 27, 28, 29]),
 array([30, 31]),
 array([61, 62]),
 array([63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]),
 array([32, 33]),
 array([34, 35, 36, 37, 38, 39, 40]),
 array([41, 42, 43, 44]),
 array([45, 46, 47, 48, 49]),
 array([50, 51, 52, 53]),
 array([54, 55, 56, 57, 58]),
 array([59, 60]),
 array([74, 75]),
 array([76, 77, 78, 79, 80, 81, 82]),
 array([83, 84, 85, 86]),
 array([87, 88, 89, 90, 91]),
 array([92, 93, 94, 95]),
 array([ 96,  97,  98,  99, 100]),
 array([101, 102])]

In [12]:
charges

[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0]

In [13]:
D_matrix_cg, q_vector_cg = constructChargegroupConstraints(
    chargeGroups = c_groups, N = N_horton, q = charges, debug=False)

In [14]:
### Same Charged Atoms
sca_by_index = read_SameChargedAtoms(
    file_name='sandbox/horton_esp_fit_constrained/Atoms_have_the_same_charges.csv',
    name=ua_names)

print(sca_by_index)

[[  3  32  74  30  59 101]
 [  4  33  75  31  60 102]
 [ 12  41  83  21  50  92]
 [ 13  42  84  22  51  93]
 [ 14  43  85  23  52  94]
 [ 15  44  86  24  53  95]
 [ 16  45  87  25  54  96]
 [ 17  46  88  26  55  97]
 [ 18  47  89  27  56  98]
 [ 19  48  90  28  57  99]
 [ 20  49  91  29  58 100]]


In [15]:
D_matrix_sym, q_vector_sym = constructPairwiseSymmetryConstraints(
    charges = sca_by_index, N = N_horton, symmetry = 1.0, debug = False)

#print D_matrix_sym, D_matrix_sym.shape
#print q_vector_sym, q_vector_sym.shape


### Concatenate The Constraints
D_ms = np.append(D_matrix_cg, D_matrix_sym, axis=0) #D_matrices
q_vs = np.append(q_vector_cg, q_vector_sym)         #q_vectors
D_matrix_all, q_vector_all = concatenated_constraints(D_matrices = D_ms,
                                                      q_vectors = q_vs)

print(D_matrix_all, D_matrix_all.shape)
print(q_vector_all, q_vector_all.shape)


[[ 1.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] (79, 103)
[ 0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.
  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.] (79,)


In [16]:
### Constrained Minimization
X, A, B = constrainedMinimize(A_matrix = A_horton,
                        b_vector = B_horton,
                        C_scalar = C_horton,
                        D_matrix = D_matrix_all,
                        q_vector = q_vector_all,
                        debug    = False)

In [18]:
print('Results:')
#prevent scientific notation and make the prints mor readable
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
print('charges {}:\n {}\ncharge sum = {}\n'.format( X[:N_horton].T.shape,
                                                    X[:N_horton].T,
                                                    X[:N_horton].T.sum() ))
print('Lagrange multipliers {}:\n {}'.format( X[N_horton:].T.shape,
                                              X[N_horton:].T ) )


### test the results
print( 'optimized result: ',
    (np.dot(X.T, np.dot(A, X)) - 2*np.dot(B.T, X) - C_horton) )

Results:
charges (103,):
 [-1.623  1.158  0.26  -1.244  1.893 -0.898  1.288 -0.48  -0.605  0.03
 -0.932  1.54  -0.148  0.336  0.541 -0.431 -0.035 -0.444  0.196  0.878
 -0.627  0.314  0.47  -0.043  0.23  -0.159 -3.052  1.402 -0.847  4.014
 -0.044  0.144 -0.125  0.341 -0.916 -0.015 -0.075 -0.619 -0.213 -0.678
  1.205 -0.251 -0.172  0.24  -0.13  -1.174 -2.535 -0.35   3.166  1.847
  0.583 -0.296 -0.313  0.326 -1.567 -2.647  3.739 -0.501  1.829 -0.266
 -0.114 -0.567  0.558 -0.21  -0.399 -0.045 -0.836 -0.67  -0.927  1.469
 -1.289  2.721 -1.369  1.513  0.256 -0.558 -1.415  1.244 -0.525  0.751
 -0.716 -0.173  0.954  0.279 -0.014 -0.065 -0.023 -1.5   -2.402  3.083
  2.835 -0.983 -0.764  0.945 -0.539  0.456 -3.787 -2.171  2.942  2.434
  1.978 -0.666 -0.782]
charge sum = 4.423581231871656

Lagrange multipliers (79,):
 [ -4.174e+03   1.940e+16  -4.254e+03  -2.513e+16   8.988e+15  -2.100e+16
  -4.079e+03  -4.215e+03  -4.116e+03  -4.094e+03  -4.201e+03  -4.193e+03
  -2.100e+16  -4.027e+03  -2.100e+1